In [1]:
experiment_name = "news_exp"

In [2]:
initial_prompt = """
You goal is to classify the following news headlines into one of the following categories:
1- World
2- Sports
3- Business
4- Sci/Tech

Provide your response as a JSON dictionary with the following structure:
{
    "chain_of_thought": "Your step-by-step reasoning here",
    "topic": "The identified topic - the ONLY possible topics are "World", "Sports", "Business", "Sci/Tech". Output just one single category."
}
"""

In [3]:
# Output format prompt
output_format_prompt = """
Provide your response as a JSON dictionary with the following structure:
{
    "chain_of_thought": "Your step-by-step reasoning here",
    "topic": "The identified topic - the ONLY possible topics are "World", "Sports", "Business", "Sci/Tech". Output just one single category."
}
"""

In [4]:
# Define output schema
output_schema = {
    'key_to_extract': 'topic',
    'value_mapping': {
        'World': 'World',
        'Sports': 'Sports',
        'Business': 'Business',
        'Sci/Tech': 'Sci/Tech',
    },
    'regex_pattern': r'"topic":\s*"([^"]+)"',  # Changed from \w+ to [^"]+ to capture everything until the closing quote
    #
    'chain_of_thought_key': 'chain_of_thought',  
    'chain_of_thought_regex': r'"chain_of_thought":\s*"(.*?)"',
    #
    'use_json_mode': True,
}

In [5]:
# Set number of optimization iterations
iterations = 2

In [6]:
# Define model providers and models for evaluation and optimization
eval_provider = "openai"
eval_model = "gpt-4o-mini"
optim_provider = "openai"
optim_model = "gpt-4o-mini"

In [7]:
# Path to the CSV file containing review data for evaluation
eval_datapath = "news.csv"
text_column = "Title"
target_column = "topic"
sample_size = 10

------------------------------------------------------------------------------------------

In [8]:
# Import necessary libraries
import pandas as pd
import sys
import os
# Add the parent directory to sys.path
# Use getcwd() to get the current working directory for Jupyter notebooks
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
grandparent_dir = os.path.dirname(parent_dir)
sys.path.append(grandparent_dir)
from src.iterative_prompt_optimization import optimize_prompt

In [9]:
# Load and prepare data
eval_data = pd.read_csv(eval_datapath, encoding='ISO-8859-1', usecols=[text_column, target_column])
eval_data = eval_data.rename(columns={text_column: 'text', target_column: 'label'})
eval_data["text"] = eval_data["text"].astype(str)
eval_data

,text,label
0,Fears for T N pension after talks,Business
1,The Race is On: Second Private Team Sets Launc...,Sci/Tech
2,Ky. Company Wins Grant to Study Peptides (AP),Sci/Tech
3,Prediction Unit Helps Forecast Wildfires (AP),Sci/Tech
4,Calif. Aims to Limit Farm-Related Smog (AP),Sci/Tech
...,...,...
7595,Around the world,World
7596,Void is filled with Clement,Sports
7597,Martinez leaves bitter,Sports
7598,5 of arthritis patients in Singapore take Bext...,Business


In [10]:
# Randomly select n samples from each class
eval_data = (
    eval_data.groupby('label')
    .apply(lambda x: x.sample(n=sample_size, random_state=42))
    .reset_index(drop=True)
)
# Shuffle the DataFrame randomly
eval_data = eval_data.sample(frac=1, random_state=42).reset_index(drop=True)
print(f"Evaluation data shape: {eval_data.shape}")
print(eval_data.head())

Evaluation data shape: (40, 2)
                                                text     label
0         Greenspan: Debt, home prices not dangerous  Sci/Tech
1  Explorers Find Ancient City in Remote Peru Jun...  Sci/Tech
2                           Countdown to Deep Impact  Sci/Tech
3                     Ali gives Iraq fighting chance    Sports
4  Lloyds TSB to Move More Than 1,000 UK Jobs to ...  Business


In [11]:
# Run the prompt optimization process
best_prompt, best_metrics = optimize_prompt(
    initial_prompt = initial_prompt,
    eval_data = eval_data,
    iterations =iterations,
    eval_provider=eval_provider,
    eval_model=eval_model,
    eval_temperature=0.7,
    optim_provider=optim_provider,
    optim_model=optim_model,
    optim_temperature=0,
    use_cache=True,
    output_format_prompt = output_format_prompt,
    output_schema=output_schema,
    fp_comments = "",
    fn_comments = "",
    tp_comments = "",
    invalid_comments="",
    validation_comments="",
    experiment_name = experiment_name,
    skip_prompt_validation = True,
)


Detected problem type: multiclass
Selected evaluation provider: openai
Selected evaluation model: gpt-4o-mini
Evaluation temperature: 0.7
Selected optimization provider: openai
Selected optimization model: gpt-4o-mini
Optimization temperature: 0
Estimated token usage: 48640
Estimated cost: $0.06

Do you want to proceed with the optimization? (Y/N): 
Iteration 1/2


╭────────────────────────────────────────────── Current Full Prompt ──────────────────────────────────────────────╮
│                                                                                                                 │
│ You goal is to classify the following news headlines into one of the following categories:                      │
│ 1- World                                                                                                        │
│ 2- Sports                                                                                                       │
│ 3- Business                                                                                                     │
│ 4- Sci/Tech                                                                                                     │
│                                                                                                                 │
│ Provide your response as a JSON dictionary with the following structure:                                        │
│ {                                                                                                               │
│     "chain_of_thought": "Your step-by-step reasoning here",                                                     │
│     "topic": "The identified topic - the ONLY possible topics are "World", "Sports", "Business", "Sci/Tech".    │
│ Output just one single category."                                                                               │
│ }                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

-----------------------------------
Processing text 1/40 .....
Using cached output for text 1/40
Prediction 1/40: Business | Ground Truth: Sci/Tech ❌ (Incorrect)
-----------------------------------
Processing text 2/40 .....
Using cached output for text 2/40
Prediction 2/40: Sci/Tech | Ground Truth: Sci/Tech ✅ (Correct)
-----------------------------------
Processing text 3/40 .....
Using cached output for text 3/40
Prediction 3/40: Sci/Tech | Ground Truth: Sci/Tech ✅ (Correct)
-----------------------------------
Processing text 4/40 .....
Using cached output for text 4/40
Prediction 4/40: World | Ground Truth: Sports ❌ (Incorrect)
-----------------------------------
Processing text 5/40 .....
Using cached output for text 5/40
Prediction 5/40: Business | Ground Truth: Business ✅ (Correct)
-----------------------------------
Processing text 6/40 .....
Using cached output for text 6/40
Prediction 6/40: Sci/Tech | Ground Truth: Sci/Tech ✅ (Correct)
-----------------------------------
Proce

Evaluation Metrics - Iteration 1
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Metric              ┃  Value ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ Precision           │ 0.8776 │
│ Recall              │ 0.8500 │
│ Accuracy            │ 0.8500 │
│ F1-score            │ 0.8544 │
│                     │        │
├─────────────────────┼────────┤
│ Valid Predictions   │     40 │
│ Invalid Predictions │      0 │
└─────────────────────┴────────┘

       Confusion Matrix        
┏━━━━━━━━━━━━━┳━━━┳━━━┳━━━┳━━━┓
┃ True↓/Pred→ ┃ 0 ┃ 1 ┃ 2 ┃ 3 ┃
┡━━━━━━━━━━━━━╇━━━╇━━━╇━━━╇━━━┩
│ 0           │ 9 │ 0 │ 0 │ 1 │
├─────────────┼───┼───┼───┼───┤
│ 1           │ 1 │ 8 │ 0 │ 1 │
├─────────────┼───┼───┼───┼───┤
│ 2           │ 0 │ 0 │ 8 │ 2 │
├─────────────┼───┼───┼───┼───┤
│ 3           │ 1 │ 0 │ 0 │ 9 │
└─────────────┴───┴───┴───┴───┘


************
Generating new prompt for multiclass classification...

Analyzing predictions for multiclass classification...


╭───────────────────────────────────────── Correct Predictions Analysis ──────────────────────────────────────────╮
│                                                                                                                 │
│ ```json                                                                                                         │
│ {                                                                                                               │
│     "analysis": {                                                                                               │
│         "common_patterns": [                                                                                    │
│             "Successful classifications often stem from identifying key terms or phrases that directly relate   │
│ to the specific category. For instance, terms like 'explorers', 'ancient city', and 'supercomputer' clearly     │
│ indicate a Sci/Tech classification, while 'jobs', 'pay cut', and 'revenue' are strong indicators of Business.", │
│             "The reasoning process frequently involves contextualizing the headline within broader themes, such │
│ as geopolitical issues for World, competitive events for Sports, and technological advancements for Sci/Tech."  │
│         ],                                                                                                      │
│         "key_features": [                                                                                       │
│             "Specific keywords are pivotal: 'explosion', 'military', and 'conflict' are strong indicators for   │
│ World; 'team', 'player', and 'match' signal Sports; 'financial', 'corporate', and 'economic' relate to          │
│ Business; and 'innovation', 'technology', and 'research' point to Sci/Tech.",                                   │
│             "The presence of well-known entities (e.g., 'Newcastle', 'IBM', 'Agassi') often helps anchor the    │
│ classification, as they are associated with specific categories."                                               │
│         ],                                                                                                      │
│         "category_boundaries": [                                                                                │
│             "World news is characterized by geopolitical events, social issues, and international relations,    │
│ while Sports is defined by competitive activities and athlete-related news. Business focuses on economic        │
│ activities, corporate decisions, and financial news, whereas Sci/Tech encompasses advancements in technology    │
│ and scientific discoveries.",                                                                                   │
│             "Clear distinctions are made based on the context of the headline; for example, a headline about a  │
│ sports event will not overlap with a business transaction, even if both involve well-known entities."           │
│         ],                                                                                                      │
│         "reasoning_strategies": [                                                                               │
│             "Effective reasoning often includes breaking down the headline into its components and assessing    │
│ the implications of each part. This method allows for a more nuanced understanding of the context and relevance │
│ to the categories.",                                                                                            │
│             "Utilizing a process of elimination is also common, where the classifier discards categories that   │
│ do not fit based on the content of the headline, leading to a more accurate classification."                    │
│         ],                                                                                                      │
│         "reinforcement_suggestions": [                

╭──────────────────────────────────────── Incorrect Predictions Analysis ─────────────────────────────────────────╮
│                                                                                                                 │
│ Here’s a concise analysis of the incorrect predictions in the multiclass classification of news headlines:      │
│                                                                                                                 │
│ 1. **Common Patterns in Misclassifications**:                                                                   │
│    - **Sports vs. World Confusion**: There is a notable pattern where headlines that could pertain to sports    │
│ are misclassified as world news. For example, "Ali gives Iraq fighting chance" and "Melvin Faces Major          │
│ Challenge in Arizona" both contain ambiguous language that could refer to either sports or geopolitical issues. │
│ This suggests a recurring confusion between sports-related contexts and broader world events.                   │
│    - **Source of Confusion**: The use of terms like "fighting chance" and "major challenge" can imply           │
│ competition or struggle, which are common in sports but can also relate to political or social issues.          │
│                                                                                                                 │
│ 2. **Ambiguous Elements Leading to Incorrect Classifications**:                                                 │
│    - **Economic Terms Misleadingly Classified**: Headlines like "Greenspan: Debt, home prices not dangerous"    │
│ and "Iraqi oil exports slump: report" illustrate how economic discussions can be misclassified. The presence of │
│ economic terminology can lead to a misinterpretation of the context, as seen in the misclassification of "Iraqi │
│ oil exports slump" as business rather than world news.                                                          │
│    - **Source of Confusion**: The overlap between economic issues and global events can create ambiguity,       │
│ especially when the economic context has significant geopolitical implications.                                 │
│                                                                                                                 │
│ 3. **Chain of Thought Errors**:                                                                                 │
│    - **Overemphasis on Specific Terms**: In several cases, the reasoning provided in the chain of thought       │
│ overly focuses on specific terms (e.g., "debt," "Iraq") without considering the broader context. For instance,  │
│ the reasoning for "China Closes 1,600 Internet Cafes in Crackdown" failed to recognize the technological        │
│ implications of internet regulation, leading to a misclassification as world news.                              │
│    - **Recurring Pattern**: The tendency to anchor the classification on prominent keywords without a holistic  │
│ view of the headline's implications can lead to errors.                                                         │
│                                                                                                                 │
│ 4. **Potential Category Boundary Confusion Points**:                                                            │
│    - **Overlap Between Business and World**: Headlines that discuss economic activities, such as oil exports or │
│ financial figures, often straddle the line between business and world news. The misclassification of "Iraqi oil │
│ exports slump: report" as business instead of world highlights this boundary confusion.                         │
│    - **Source of Confusion**: The interconnectedness of global economics and politics can blur the lines        │
│ between these categories, making it challenging to classify headlines accurately.                               │
│                                                       

Number of samples: 40


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''